### Imports

In [1]:
import pandas as pd
import numpy as np
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import sklearn.mixture as mix

import matplotlib.pyplot as plt

# Efficiency Testing Libraries
from statsmodels.tsa.stattools import bds
from statsmodels.sandbox.stats.runs import runstest_1samp
from statsmodels.tsa.stattools import adfuller
import scipy.stats as sps
import yfinance

### Data Extraction

In [2]:
# Data Extraction
start_date = "2017-01-1"
end_date = "2023-06-1"
symbol = "SPY"

df = yfinance.download(tickers = "SUZLON.NS",start="2000-03-06",
                               interval = "1d", group_by = 'ticker', auto_adjust = True)

df = df[["Open", "High", "Low", "Close","Volume"]]

df.head()
df["Returns"] = df["Close"] / df["Close"].shift(1) - 1
df.dropna(inplace=True)
df.head(3)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume,Returns
Date,,,,,,
2005-10-20,138.319246,139.651142,125.690959,127.644402,39078530,-0.066320
2005-10-21,125.345658,137.135349,125.345658,132.962097,25623135,0.041660
2005-10-24,135.556822,136.148779,127.920661,129.035492,11125060,-0.029532


In [3]:
# Returns extraction
returns = df["Returns"].values.astype(float)

### Runs Test

Base standard test for randomness based on linearity

In [4]:
# Convert Returns into binary outcomes
returns_binary = [ 1 if x >= 0 else 0 for x in returns]

In [5]:
(z_stat, p_value) = runstest_1samp(returns_binary[:10], correction=False)
z_stat = round(z_stat, 3)
p_value = round(p_value, 3)
is_reject_runs = True if p_value < 0.05 else False
print(f"Z-Statistic a statistical measurement that describes a value's relationship to the mean of a group of values. 0 near to mean: {z_stat}")
print(f"P-Value: {p_value}")
print(f"Reject Null: {is_reject_runs}")
print(f"Observable Runs Exceeds Excpected Runs by: {z_stat} Standard Deviations")
print("Not Random") if is_reject_runs else print("Random")

Z-Statistic a statistical measurement that describes a value's relationship to the mean of a group of values. 0 near to mean: 0.14
P-Value: 0.888
Reject Null: False
Observable Runs Exceeds Excpected Runs by: 0.14 Standard Deviations
Random


### BDS Test

Testing for chaos and nonlinearity. Considered as your last line of defence as takes into account non-linear dependancies after running other efficiency tests.

In [6]:
bds_test = bds(returns[-500:], distance=2)
bds_stat = float(bds_test[0])
pvalue = float(bds_test[1])
print("BDS Test Statistic: ", round(bds_stat, 3))
print("BDS P-Value: ", round(pvalue, 3))
print("Not Random") if pvalue < 0.05 else print("Random")

BDS Test Statistic:  2.605
BDS P-Value:  0.009
Not Random


### Hurst Exponent

"Whether a market tends to trend, mean revert, or is just random is valuable information for a trader. While the Hurst exponent isn't an entry signal in and of itself, it can serve as a filter on top of a system. Given that market regimes can shift over time to favor one approach or the other, overlaying your model with a Hurst filter could help prevent your algorithm from buying a breakout in a mean reverting market or shorting ahead of a pullback when the market is moving to new highs." - *Find Your Best Market to Trade With the Hurst Exponent (referenced below)*

If Hurst = 0.5, then the market is random.

If Hurst > 0.5, then there is evidence of a trending market.

If Hurst < 0.5, then there is evidence of a mean reverting market.

In [7]:
def hurst(ts, min_lag=1, max_lag=100):
    lags = range(min_lag, max_lag)
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]
    poly = polyfit(log(lags), log(tau), 1)
    return poly[0]*2.0

In [10]:
prices = df["Close"].values
hurst_res = hurst(prices)
status = "sidewise"
if(hurst_res < 0.5):
    status = "DownTrend"
else:
    status = "UpTrend"
print("market Trend :",status,",hurst value :",hurst_res)

market Trend : UpTrend ,hurst value : 0.5204681692826308


### AD Fuller Test for Stationarity

In [9]:
dftest = adfuller(returns)
p_value = dftest[1]
t_test = dftest[0] < dftest[4]["1%"]
print(p_value, t_test)
print("If < 0.05 and True then we can reject the null hypothesis and conclude that the index is stationary")

0.0 True
If < 0.05 and True then we can reject the null hypothesis and conclude that the index is stationary


### Resources and Useful References

NEDL YouTube Channel - Hurst Exponent: https://www.youtube.com/watch?v=l08LICz8Ink

NEDL YouTube Channel - Dynamic Hurst Exponent: https://www.youtube.com/watch?v=v0sivj2wGcA

Hurst Exponent Coding: https://raposa.trade/blog/find-your-best-market-to-trade-with-the-hurst-exponent/

More Hurst Exponent Coding: https://www.quantstart.com/articles/Basics-of-Statistical-Mean-Reversion-Testing/